In [1]:
import csv
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
Gestures = ['Come','Stop','NG','Right','Left']
GesturesFull = ['Pulling_Hand_In','Stop_Sign','No_gesture','Swiping_Left','Swiping_Right']
GesturesPaper = ['Go Straight','Stop','No Gesture','Go Right','Go Left']

In [2]:
def GetData(sc,gest):
    with open("data/Scen"+str(sc)+Gestures[gest]+".csv") as f:
        reader = csv.reader(f)
        data = []
        for row in reader:
            data.append(row)
    return data

In [3]:
def SingleGestureAccList(datarr,gest):
    newarr = []
    for itn in range(1,2002):
        start = itn*26 - 26
        end = start + 25
        itnarray=datarr[start:end+1]
        foundgesture=False
        for item in itnarray:
            if item[1] == GesturesFull[gest]:
                newarr.append(float(item[2]))
                foundgesture=True
        if foundgesture == False:
            newarr.append(0.0006)
    return newarr

In [4]:
def SummedData(datarr): #Adds the percentages of all 5 gestures in that iteration (of 2000)
    ComeList = SingleGestureAccList(datarr,0)
    StopList = SingleGestureAccList(datarr,1)
    NGList = SingleGestureAccList(datarr,2)
    RightList = SingleGestureAccList(datarr,3)
    LeftList = SingleGestureAccList(datarr,4)
    addnormalizedarr = []
    for i in range(0,2001):
        addnormalizedarr.append(ComeList[i]+StopList[i]+NGList[i]+RightList[i]+LeftList[i])
    
    return addnormalizedarr

In [5]:
def GetScore(datarr,gest,thresh):
    SumsPerITN = SummedData(datarr)
    count=0
    iteration = 0
    for item in datarr:
        if item[1] == GesturesFull[gest]:
            #print(str(float(item[2]))+" "+str(SumsPerITN[iteration]))
            normalizedacc = float(item[2])/SumsPerITN[iteration]
            if normalizedacc > thresh:
                count+=1
            iteration+=1
    return count

In [6]:
def CountTopAppearances(datarr,gest):
    count=0
    for itn in range(1,2001):
        start = itn*26 - 26
        end = start + 25
        itnarray=datarr[start:end+1]
        for item in itnarray:
            if item[1] in GesturesFull:
                if item[1] == GesturesFull[gest]:
                    count+=1
                break
    return count

In [7]:
def GetCount(scen, gest):
    return CountTopAppearances(GetData(scen,gest),gest)

In [8]:
def GetTopTable(sc,gest,thresh):
    datarr=GetData(sc,gest)
    countcorrect = GetScore(datarr,gest,thresh)
    countincorrect = 2000 - countcorrect
    return float(countcorrect),float(countincorrect)

In [9]:
def GetBottomTable(sc,gest,thresh):
    total=2000
    countofgest=0
    if gest != 0:
        ComeDatArr = GetData(sc,0)
        countofgest += GetScore(ComeDatArr,gest,thresh)
        total += 2000
    if gest != 1:
        StopDatArr = GetData(sc,1)
        countofgest += GetScore(StopDatArr,gest,thresh)
        total += 2000
    if gest != 2:
        NGDatArr = GetData(sc,2)
        countofgest += GetScore(NGDatArr,gest,thresh)
        total += 2000
    if gest != 3 and sc==1:
        RightDatArr = GetData(sc,3)
        countofgest += GetScore(RightDatArr,gest,thresh)
        total+=2000
    if gest != 4 and sc==1:
        LeftDatArr = GetData(sc,4)
        countofgest += GetScore(LeftDatArr,gest,thresh)
        total+=2000
    nocountofgest=total-countofgest
    return float(countofgest),float(nocountofgest)

In [12]:
def GetTablePR(sc,gest,thresh,metric=0):
    TP,FN=GetTopTable(sc,gest,thresh)
    FP,TN=GetBottomTable(sc,gest,thresh)
    try:
        Precision = TP/(TP+FP)
    except ZeroDivisionError:
        Precision = 0
    try:
        Recall = TP/(TP+FN)
    except ZeroDivisionError:
        Recall = 0
    
    if metric == 0:
        return Precision, Recall
    if metric == 1:
        if sc==1:
            crashrate=float(FN+FP)/10000
        else:
            crashrate=float(FN+FP)/6000
        return crashrate
    if metric == 2:
        trueaccuracy = (TP+TN)/(TP+TN+FN+FP)
        return trueaccuracy
    if metric == 3:
        f1 = (2*Precision*Recall)/(Precision+Recall)
        return f1

In [11]:
def GetCurve(sc,threshlist):
    baseline = (1.0/3.0)
    if sc==1: 
        gestlist=[0,1,2,3,4]
        baseline = 0.2
    else:
        gestlist=[0,1,2]
    colors = ['r','b','g','y','c']
    gesturenum=0
    fig, ax = plt.subplots()
    for gest in gestlist:
        PrArr=[]
        ReArr=[]
        for thresh in threshlist:
            Pr,Re=GetTablePR(sc,gest,thresh)
            PrArr.append(Pr)
            ReArr.append(Re)
        with open('graphdata.csv', "w") as csv_file:
            writ = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
            writ.writerow(PrArr)
            writ.writerow(ReArr)
        ax.plot(ReArr, PrArr, str(colors[gesturenum])+'-',label=str(GesturesPaper[gest]))
        gesturenum+=1
    ax.axhline(y=baseline, color='black', linestyle='--',label='Baseline')
    plt.axis([0, 1.01, 0, 1.01])
    plt.title('Scenario '+str(sc))
    handles, labels = ax.get_legend_handles_labels()
    ax.legend(labels,loc=6)
    ax.set_ylabel('Precision')
    ax.set_xlabel('Recall')
    
    plt.savefig('Scenario'+str(sc)+'.png',dpi=2000)
    plt.show()

In [12]:
def ThreshGen(NumPoints):
    diff = 1.0/float(NumPoints)
    arr = [0]
    for point in range(1,NumPoints):
        arr.append(point*diff)
    return arr

In [15]:
def GetGraphs(NumPoints): # Generate Precision Recall Graphs
    GetCurve(1,ThreshGen(500))
    GetCurve(2,ThreshGen(500))
    GetCurve(3,ThreshGen(500))
    GetCurve(4,ThreshGen(500))

In [35]:
def BenchmarkMetrics(metric=0): #Get Comparison Metrics: 0=PR,1=CrashRate,2=Accuracy,3=F1
    print("Scenario 1")
    S11 = GetTablePR(1,0,0.4,metric)
    print('\t' + Gestures[0] + ': ' + str(S11))
    S12 = GetTablePR(1,1,0.4,metric)
    print('\t' + Gestures[1] + ': ' + str(S12))
    S13 = GetTablePR(1,2,0.4,metric)
    print('\t' + Gestures[2] + ': ' + str(S13))
    S14 = GetTablePR(1,3,0.4,metric)
    print('\t' + Gestures[3] + ': ' + str(S14))
    S15 = GetTablePR(1,4,0.4,metric)
    print('\t' + Gestures[4] + ': ' + str(S15))
    
    print("Scenario 2")
    S21 = GetTablePR(2,0,0.4,metric)
    print('\t' + Gestures[0] + ': ' + str(S21))
    S22 = GetTablePR(2,1,0.4,metric)
    print('\t' + Gestures[1] + ': ' + str(S22))
    S23 = GetTablePR(2,2,0.4,metric)
    print('\t' + Gestures[2] + ': ' + str(S23))
    
    print("Scenario 3")
    S31 = GetTablePR(3,0,0.4,metric)
    print('\t' + Gestures[0] + ': ' + str(S31))
    S32 = GetTablePR(3,1,0.4,metric)
    print('\t' + Gestures[1] + ': ' + str(S32))
    S33 = GetTablePR(3,2,0.4,metric)
    print('\t' + Gestures[2] + ': ' + str(S33))
    
    print("Scenario 4")
    S41 = GetTablePR(4,0,0.4,metric)
    print('\t' + Gestures[0] + ': ' + str(S41))
    S42 = GetTablePR(4,1,0.4,metric)
    print('\t' + Gestures[1] + ': ' + str(S42))
    S43 = GetTablePR(4,2,0.4,metric)
    print('\t' + Gestures[2] + ': ' + str(S43))
    
    print("Overall Average of 14")
    print('\t' + str(float(S11+S12+S13+S14+S15+S21+S22+S23+S31+S32+S33+S41+S42+S43)/14))

In [37]:
BenchmarkMetrics(2)

Scenario 1
	Come: 0.934166666667
	Stop: 0.955833333333
	NG: 0.933166666667
	Right: 0.882083333333
	Left: 0.950333333333
Scenario 2
	Come: 0.94925
	Stop: 0.932625
	NG: 0.98125
Scenario 3
	Come: 0.936125
	Stop: 0.911625
	NG: 0.964125
Scenario 4
	Come: 0.923375
	Stop: 0.98725
	NG: 0.9985
Overall Average of 14
	0.945693452381
